#### Introduction

In this project, I investigate and illustrate the usage and purpose of data transformation techniques.
While data transformation covers a wide range of area such as general standardization and normalization, up to some very technical domain such as principal component analysis. This project intended to focus on methods that change data instead of features.

The dataset employed is California Housing Prices provided by Cam Nugent on Kaggle

link to dataset: https://www.kaggle.com/camnugent/california-housing-prices

Below listed some data transformation techniques:

1.  Discretization
2.  Normalization - Min-max normalization
3.  Normalization - Decimal Scaling (ommited, just divide all value to its decimal level eg. when max = 4865, scale = divide all values by 10000 which gives 0.4865 as maximum)
4.  Normalization - Z-score normalization
5.  Max Abs Scaler
6.  Robust Scaler
7.  Quantile Transformer Scaler (ommited,another similar robust scaler)
8. Power Transformer Scaler
  
#### References:
1. https://www.upgrad.com/blog/methods-of-data-transformation-in-data-mining/
2. https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/
3. https://towardsdatascience.com/all-about-feature-scaling-bcc0ad75cb35


---
---
#### Coding
---
#### Preparation Works

In [106]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

In [2]:
# cnames = ['age','workclass','fnlwgt','education','education-num',
#           'marital-status','occupation','relationship','race','sex',
#          'capital-gain','capital-loss','hours-per-week','native-country']
df = pd.read_csv('housing.csv', index_col = False)
df.shape

(20640, 10)

In [3]:
df.head(8)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY


---
#### Discretization

Discretization refers to the process of converting continuous variables into categorical one, often ordinal. The purpose is to reduce complexity of the dataset, sometimes to ease machine learning, but mostly to ease human understanding and increase data presentability. Below I selected housing_median_age and implemented Discretization.

In [4]:
housing_age = df['housing_median_age']
housing_age.min()

1.0

In [5]:
housing_age.max()

52.0

In [6]:
housing_age.median()

29.0

##### Long and raw Approach
Distribute datapoints into number of bins based on quartile

In [7]:
housing_age.quantile([0.25,0.5,0.75])

0.25    18.0
0.50    29.0
0.75    37.0
Name: housing_median_age, dtype: float64

In [8]:
def discretizing(x):
    if x <= 18: return 1
    if x <= 29: return 2
    if x <= 37: return 3
    if x > 37: return 4

housing_age_discretized = housing_age.apply(discretizing)
housing_age_discretized.value_counts()

2    5343
1    5326
3    5026
4    4945
Name: housing_median_age, dtype: int64

##### Short Approach

Use pandas.cut()
simpler and much balance bin size, though it is also achievable though bin calculation if coding in raw.

In [9]:
housing_age_discretized_2 = pd.cut(housing_age,4)
housing_age_discretized_2

0        (39.25, 52.0]
1        (13.75, 26.5]
2        (39.25, 52.0]
3        (39.25, 52.0]
4        (39.25, 52.0]
             ...      
20635    (13.75, 26.5]
20636    (13.75, 26.5]
20637    (13.75, 26.5]
20638    (13.75, 26.5]
20639    (13.75, 26.5]
Name: housing_median_age, Length: 20640, dtype: category
Categories (4, interval[float64]): [(0.949, 13.75] < (13.75, 26.5] < (26.5, 39.25] < (39.25, 52.0]]

#### Normalization - min-max normalization

This method convert all scores to have minimum value of 0 and maximum value of 1.
Lets apply min-max normalization to median_income.

In [30]:
mscale = MinMaxScaler() 
minmax_median_income = mscale.fit_transform(df[['median_income']])
minmax_median_income = minmax_median_income.flatten()
minmax_median_income

array([0.53966842, 0.53802706, 0.46602805, ..., 0.08276438, 0.09429525,
       0.13025338])

#### Normalization - z-score Normalization

This method uses a normal distribution z-value, which convert the entire dataset to have standard deviation of 1 and mean of 0.

In [92]:
sscale = StandardScaler()
z_score_median_income = sscale.fit_transform(df[['median_income']])
z_score_median_income = z_score_median_income.flatten()
z_score_median_income

0.9999999999999999

Table below compare the original values, minmax scaler and z-score scaler.
Zscore method provides most information at a glance where if the value is near 0, then its around mean, if its negative, its below mean and vice-versa. the wider the value spread beyond or behind the mean, the sparser the data is. 
However, if deviation is the concern, minmax scaler provide better image as std() of every z-score normalized data is always 1.

In [91]:
normalized_table = pd.DataFrame()
normalized_table['income'] = df['median_income']
normalized_table['minmax_income'] = minmax_median_income
normalized_table['zscore_income'] = z_score_median_income
normalized_table.head(10)

,income,minmax_income,zscore_income
0,8.3252,0.539668,2.344766
1,8.3014,0.538027,2.332238
2,7.2574,0.466028,1.782699
3,5.6431,0.354699,0.932968
4,3.8462,0.230776,-0.012881
5,4.0368,0.243921,0.087447
6,3.6591,0.217873,-0.111366
7,3.1200,0.180694,-0.395137
8,2.0804,0.108998,-0.942359
9,3.6912,0.220087,-0.094470


#### Max Abs Scaler

This method convert the data by divide all values by its max value. In another word, this method will result in data.max() = 1. It works similarily to minmaxscaler but do not destroy the original sparcity information of the dataset.

In [114]:
mascaler = MaxAbsScaler()
ma_median_income =  mascaler.fit_transform(df[['median_income']])
ma_median_income = ma_median_income.flatten()
print(ma_median_income.max())
print(ma_median_income.min())

1.0
0.03332644449037007


#### Robust Scaler

If outlier is an issue and cannot be removed, robustscaler is the way to go, instead of using mean value as the center as in z-score normalization, this method employes median and quantiles. It removes the median values from all data and scale the data according to the quantile range.

In [115]:
rscaler = RobustScaler()
r_median_income = rscaler.fit_transform(df[['median_income']])
r_median_income = r_median_income.flatten()
print(r_median_income.max())
print(r_median_income.min())

5.259673830768172
-1.392251760442232


#### Power Transformer
if normality is an issue, power transformer can make data more Gaussian-like
do note it breaks the original sparcity information. as shown below, both the min and max are around the same value having mean value around 0 implying a bell shaped distribution

In [116]:
ptscaler = PowerTransformer()
pt_median_income = ptscaler.fit_transform(df[['median_income']])
pt_median_income = pt_median_income.flatten()
print(pt_median_income.max())
print(pt_median_income.min())
print(pt_median_income.mean())

3.146571508664235
-3.4460847380774338
-9.363741479009072e-17


This project investigated the methods of data transformation as well as its usage and purposes. While some seems similar, each of those are only applicable and useful in distinct circumstances. During preprocessing, it is important to identify the method of transformation, which had been proven to improve ml performance (both accuracy and runtime).
Below I summarized the original value and all the transformed values as well as providing the data summary.



In [134]:
finaldf = pd.DataFrame()
finaldf['original income'] =  df['median_income']
finaldf['minmax'] = minmax_median_income
finaldf['zscore'] = z_score_median_income
finaldf['maxabs'] = ma_median_income
finaldf['robust'] = r_median_income
finaldf['powertr'] = pt_median_income 
finaldf.head()

,original income,minmax,zscore,maxabs,robust,powertr
0,8.33,0.54,2.34,0.56,2.20,1.90
1,8.30,0.54,2.33,0.55,2.19,1.90
2,7.26,0.47,1.78,0.48,1.71,1.60
3,5.64,0.35,0.93,0.38,0.97,1.05
4,3.85,0.23,-0.01,0.26,0.14,0.21


In [132]:
#supress scientific notation by round off value to 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

#summary statistics
summary = finaldf.agg({ "original income": ['min','max','mean','median','skew'],
                       'minmax' : ['min','max','mean','median','skew'],
                       'zscore' : ['min','max','mean','median','skew'],
                       'maxabs' : ['min','max','mean','median','skew'],
                       'robust' : ['min','max','mean','median','skew'],
                       'powertr' : ['min','max','mean','median','skew']
                      })
summary

,original income,minmax,zscore,maxabs,robust,powertr
min,0.50,0.00,-1.77,0.03,-1.39,-3.45
max,15.00,1.00,5.86,1.00,5.26,3.15
mean,3.87,0.23,0.00,0.26,0.15,-0.00
median,3.53,0.21,-0.18,0.24,0.00,0.02
skew,1.65,1.65,1.65,1.65,1.65,-0.00
